## Helper functions
Here we define some functions which we use in `compute_bitangents.ipynb` to compute bitangents of symmetric smooth planar quartics and their symmetries

In [3]:
from sage.symbolic.expression_conversions import polynomial

# Round complex numbers
def round_cx(z0,n):
    return float(round(z0.real(),n)) + float(round(z0.imag(),n))*I

# The following inputs a line ax+by+cz and outputs the tuple of coefficients [a,b,c]
def line_to_coeffs(bitangent):
    return [CC(bitangent.coefficient({x:1})), CC(bitangent.coefficient({y:1})),CC(bitangent.coefficient({z:1}))]

# Turn a list of three coefficients [a,b,c] into the polynomial ax+by+cz=0
def coeffs_to_line(list_of_coeffs):
    return x*list_of_coeffs[0] + y*list_of_coeffs[1] + z*list_of_coeffs[2]

# Provide a normalized version of a bitangent
def normalize(bitangent,numzeros=4):
    # Base change to CC
    b = bitangent.change_ring(ComplexField(30))

    # If the x-coordinate is nonzero mod out by it
    if line_to_coeffs(b)[0] != 0:
        c = b*(1/line_to_coeffs(b)[0])
    # Else mod out by the y-coordinate
    elif line_to_coeffs(b)[1] != 0:
        c = b*(1/line_to_coeffs(b)[1])
    # Or the z-coordinate
    else:
        c = b*(1/line_to_coeffs(b)[2])
    
    # Round the output coordinates and return a new line
    new_coeffs = [round_cx(a,numzeros) for a in line_to_coeffs(c)]
    return coeffs_to_line(new_coeffs)

# Returns all bitangents of a quartic in normalized form
def compute_normalized_bitangents(f):
    return [normalize(b) for b in compute_bitangents(f)]

We write some functions to make the equations of bitangents more readable and act on them via a subgroup $G\le \text{PGL}_3(\mathbb{C})$.

In [ ]:
# Apply a matrix M to a line B = {ax+by+cy=0}
def gp_action(M,B,numzeros=4):
    vec = matrix([[B.coefficient({x:1})],[B.coefficient({y:1})],[B.coefficient({z:1})]])
    output_mat = (M*vec).transpose()
    w = list(list(output_mat)[0])
    outputline = w[0]*x + w[1]*y + w[2]*z
    return normalize(outputline,numzeros)

# Returns a list of rows of a matrix
def matrix_to_list(M):
    return [list(r) for r in M.rows()]

# Generates a candidate list of the orbit of an input bitangent under a list of group elements
def candidate_orbit(bitangent,list_of_gp_elements):
    output_list = [bitangent]
    isotropy_gens = []
    for M in list_of_gp_elements:
        output_list.append(gp_action(M,bitangent))
        if gp_action(M,bitangent) == bitangent:
            isotropy_gens.append(M)
    for line in output_list:
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

# Trims the equation of the bitangnt, rounding coefficients to 3 digits
def trim(bitangent):
    x_coeff = CC(bitangent.coefficient({x:1}));
    y_coeff = CC(bitangent.coefficient({y:1}));
    z_coeff = CC(bitangent.coefficient({z:1}));
    return round_cx(x_coeff,3)*x + round_cx(y_coeff,3)*y + round_cx(z_coeff,3)*z

# Take the difference in absolute value between each of the coefficients and sum these
def bitan_difference(bitan1,bitan2):
    return abs(CC(bitan1.coefficient({x:1})) - CC(bitan2.coefficient({x:1}))) + abs(CC(bitan1.coefficient({y:1})) - CC(bitan2.coefficient({y:1}))) + abs(CC(bitan1.coefficient({z:1})) - CC(bitan2.coefficient({z:1})))
# This should return some indicator of how far off two equations are numerically. Uncomment and run the following for instance:
# bitan1 =x + (-1.1234 - 4.9221*I)*y + (-2.9252 - 1.4087*I)*z
# bitan2 =x + (-1.1231 - 4.9225*I)*y + (-2.9254 - 1.4091*I)*z
# bitan_difference(bitan1,bitan2)

# Boolean to check if two bitangents agree (up to some error rate)
def same_bitangent(bitan1,bitan2, error_rate=0.01):
    if bitan_difference(bitan1,bitan2) < error_rate:
        return true
    else:
        return false

# Computes the orbits of bitangents of a quartic equation
def orbits_of_bitangents(quartic_eqn,gp,threshhold=0.01):
    list_of_bitangents = compute_normalized_bitangents(quartic_eqn)
    
    orbits_list = []
    
    # For each bitangent in the list
    for bi in list_of_bitangents:

        # Build a new orbit
        orbit = [bi]

        # Act on it via the elements of the group
        for g in gp:
            acted_bitangent = gp_action(g,bi)
            
            # A priori we found a new one, 
            is_new_bitangent = True
    
            # we should check numerically we haven't already found it in this orbit though
            for bitan in orbit:
                if same_bitangent(bitan,acted_bitangent,threshhold):
                    is_new_bitangent = False
                    break

            # If we found a new bitangent, we add it to the running orbit,
            if is_new_bitangent:
                orbit.append(acted_bitangent)
                # and we delete it from the list
                for listbi in list_of_bitangents:
                    if same_bitangent(listbi,acted_bitangent,threshhold):
                        list_of_bitangents.remove(listbi)
                
        # Now add the orbit to the list of all orbits
        orbits_list.append(orbit)
    return orbits_list

# Returns the GAP id of a group
def get_gap_id(grp):
    return gap.eval("IdSmallGroup (" + str(grp.gap()) + ");")

# Returns the isotropy subgroup of an orbit of lines under a group action
def isotropy(orbit,grp):
    bi = orbit[0]
    isotropy_elts = []
    for g in grp:
        acted_bitangent =  gp_action(g,bi)

        if same_bitangent(bi,acted_bitangent):
            isotropy_elts.append(g)

    return MatrixGroup(isotropy_elts)

# Boolean, returns whether two subgroups of G are subconjugate
def are_conjugate_subgps(ambientgrp,subgp1,subgp2):
    gap.eval("g:=" + str(ambientgrp.gap()) + ";")
    gap.eval("h1:=" + str(subgp1.gap()) + ";")
    gap.eval("h2:=" + str(subgp2.gap()) + ";")
    return gap.eval("h1 in ConjugateSubgroups(g,h2);")

# Outputs the orbit of a single bitangent.
# This isn't used, but may be helpful
def orbit_of(bitangent,gp,numzeros=4):
    output = [bitangent]

    # For each element in the group, act on the original bitangent
    for g in gp:
        acted_bitangent = gp_action(g,bitangent,numzeros)
        
        # A priori we found a new one, 
        is_new_bitangent = True

        # we should check numerically we haven't already found it though
        for bi in output:
            if same_bitangent(bi,acted_bitangent,0.1):
                is_new_bitangent = False
                break

        if is_new_bitangent:
            output.append(acted_bitangent)
    return output